In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## क्लाइन्ट बनाउँदै 

यस नमुनामा, हामी [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) प्रयोग गर्नेछौं LLM पहुँचका लागि। 

`model` लाई `gpt-4o-mini` भनेर परिभाषित गरिएको छ। GitHub Models बजारमा उपलब्ध अर्को मोडेलमा परिवर्तन गरेर फरक नतिजाहरू हेर्न प्रयास गर्नुहोस्। 

छिटो परीक्षणको लागि, हामी केवल एउटा साधारण प्रम्प्ट चलाउनेछौं - `What is the capital of France`।


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## कार्यहरू परिभाषित गर्दै 

यस उदाहरणमा, हामी एजेन्टलाई एउटा उपकरणको पहुँच दिनेछौं, जुन उपलब्ध छुट्टी गन्तव्यहरूको सूची र तिनीहरूको उपलब्धता भएको कार्य हो। 

तपाईं यसलाई यस्तो परिदृश्यको रूपमा सोच्न सक्नुहुन्छ जहाँ यात्रा एजेन्टसँग, उदाहरणका लागि, यात्रा डेटाबेसको पहुँच हुन सक्छ। 

यस नमूना अध्ययन गर्दा, एजेन्टले कल गर्न सक्ने नयाँ कार्यहरू र उपकरणहरू परिभाषित गर्न प्रयास गर्न स्वतन्त्र महसुस गर्नुहोस्।


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## फङ्सन टुल परिभाषित गर्दै  
एजेन्टले `vacation_destinations` लाई `FunctionTool` को रूपमा प्रयोग गर्नको लागि, हामीले यसलाई त्यसरी नै परिभाषित गर्नुपर्छ।  

हामी टुलको विवरण पनि प्रदान गर्नेछौं, जसले एजेन्टलाई प्रयोगकर्ताले अनुरोध गरेको कार्यसँग सम्बन्धित रूपमा उक्त टुल केका लागि प्रयोग गरिन्छ भनेर पहिचान गर्न मद्दत गर्दछ।  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## एजेन्ट परिभाषित गर्दै 

अब हामी तलको कोडमा एजेन्ट बनाउन सक्छौं। हामी `system_message` परिभाषित गर्छौं ताकि एजेन्टले प्रयोगकर्ताहरूलाई छुट्टी गन्तव्यहरू खोज्न सहयोग गर्ने निर्देशन दिन सकोस्। 

हामीले `reflect_on_tool_use` पैरामीटरलाई true मा सेट गर्छौं। यसले LLM लाई टूल कलको प्रतिक्रिया लिन र प्राकृतिक भाषामा प्रतिक्रिया पठाउन अनुमति दिन्छ। 

तपाईंले यो पैरामीटरलाई false मा सेट गरेर फरक देख्न सक्नुहुन्छ।


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## एजेन्ट चलाउने 

अब हामी एजेन्टलाई सुरुमा प्रयोगकर्ताको सन्देशसँग चलाउन सक्छौं, जसले टोकियोको यात्राको लागि सोधिरहेको छ। 

तपाईं यो सहरको गन्तव्य परिवर्तन गरेर हेर्न सक्नुहुन्छ कि एजेन्टले सहरको उपलब्धतामा कसरी प्रतिक्रिया दिन्छ।


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
